In [2]:
import torch
import torch.nn as nn
import numpy as np 
import torchvision

ModuleNotFoundError: No module named 'torchvision'

In [3]:
class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(10, 20)
        self.linear2 = nn.Linear(20, 3)

    def forward(self, x):
        x = self.linear1(x)
        x = nn.functional.relu(x)
        x = self.linear2(x)
        x = nn.functional.relu(x)
        return x 
    

In [ ]:
import torch
from LRPModel import LRPModel
x = torch.rand(size=(1, 3, 224, 224))
model = torchvision.models.vgg16(pretrained=True)
lrp_model = LRPModel(model)
r = lrp_model.forward(x)

torch.Size([200]) torch.Size([20]) torch.Size([10])


RuntimeError: inconsistent tensor size, expected tensor [10] and src [200] to have the same number of elements, but got 10 and 200 elements respectively

In [ ]:

def LRP_linear(model, X, T):
    """
    X : input 
    T : Target
    """
    layers = [module for module in model.modules() if isinstance(module, nn.Linear)]
    W = [l.weight for l in layers]
    B = [l.bias for l in layers]
    L = len(W)

    # Activation
    A = [X]+[None]*L
    for l in range(L):
        print(W[l].size(), B[l].size(), A[l].size())
        A[l+1] = np.maximum(0,A[l].dot(W[l])+B[l])

    # LRP 
    R = [None]*L + [A[L]*(T[:,None]==np.arange(10))]
    for l in range(1,L)[::-1]:

        w = rho(W[l],l)
        b = rho(B[l],l)
        
        z = incr(A[l].dot(w)+b,l)    # step 1
        s = R[l+1] / z               # step 2
        c = s.dot(w.T)               # step 3
        R[l] = A[l]*c                # step 4

    # Last Pixel Layer
    w  = W[0]
    wp = np.maximum(0,w)
    wm = np.minimum(0,w)
    lb = A[0]*0-1
    hb = A[0]*0+1

    z = A[0].dot(w)-lb.dot(wp)-hb.dot(wm)+1e-9        # step 1
    s = R[1]/z                                        # step 2
    c,cp,cm  = s.dot(w.T),s.dot(wp.T),s.dot(wm.T)     # step 3
    R[0] = A[0]*c-lb*cp-hb*cm                         # step 4
    return R    